# Basic Optimization
In this example, we'll be performing a simple optimization of single-objective functions using the global-best optimizer in `pyswarms.single.GBestPSO` and the local-best optimizer in `pyswarms.single.LBestPSO`. This aims to demonstrate the basic capabilities of the library when applied to benchmark problems.

In [1]:
# Import modules
import numpy as np

# Import PySwarms
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Optimizing a function
First, let's start by optimizing the sphere function. Recall that the minima of this function can be located at `f(0,0..,0)` with a value of `0`. In case you don't remember the characteristics of a given function, simply call `help(<function>)`.

For now let's just set some arbitrary parameters in our optimizers. There are, at minimum, three steps to perform optimization:

1. Set the hyperparameters to configure the swarm as a `dict`.
2. Create an instance of the optimizer by passing the dictionary along with the necessary arguments.
3. Call the `optimize()` method and have it store the optimal cost and position in a variable.

The `optimize()` method returns a `tuple` of values, one of which includes the optimal cost and position after optimization. You can store it in a single variable and just index the values, or unpack it using several variables at once.

In [2]:
%%time
# Set-up hyperparameters
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=2, options=options)

# Perform optimization
cost, pos = optimizer.optimize(fx.sphere, iters=1000)

2021-11-13 11:37:47,950 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=8.74e-46
2021-11-13 11:37:49,988 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 8.74461468617922e-46, best pos: [-3.34220881e-24  2.93818160e-23]


Wall time: 2.05 s


We can see that the optimizer was able to find a good minima as shown above. You can control the verbosity of the output using the `verbose` argument, and the number of steps to be printed out using the `print_step` argument.

Now, let's try this one using local-best PSO:

In [3]:
%%time
# Set-up hyperparameters
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9, 'k': 2, 'p': 2}

# Call instance of PSO
optimizer = ps.single.LocalBestPSO(n_particles=10, dimensions=2, options=options)

# Perform optimization
cost, pos = optimizer.optimize(fx.sphere, iters=1000)

2021-11-13 11:37:50,116 - pyswarms.single.local_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 2, 'p': 2}
pyswarms.single.local_best: 100%|██████████|1000/1000, best_cost=5.36e-42
2021-11-13 11:37:52,671 - pyswarms.single.local_best - INFO - Optimization finished | best cost: 5.360047939107018e-42, best pos: [-1.32499769e-21  1.89853340e-21]


Wall time: 2.57 s


## Optimizing a custom objective function

Above we optimized `fx.sphere`, one of the built-in objective functions. But most users will want to optimize their own objective function. The most important thing to see here is that in PySwarms, the objective function must accept a `numpy.ndarray` with shape `(n_particles, dimensions)`. In this array, each row represents a particle, and each column represents its position on a specific dimension of the search-space. The objective function must then return an array of shape `(n_particles,)`, with one objective value for each particle.

In [4]:
def my_sphere(x):
    return np.sum(x**2, axis=1)

options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=2, options=options)
cost, pos = optimizer.optimize(my_sphere, iters=1000)

2021-11-13 11:37:52,785 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=1.1e-41
2021-11-13 11:37:54,924 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.099173922420955e-41, best pos: [ 3.15725382e-21 -1.01167561e-21]


Notice that `np.sum(x**2)` would be incorrect, as it would return a single objective value, not one per particle. Using `np.sum(x**2, axis=1)` will sum across rows only, returning one value per row (i.e. per particle).

## Optimizing a function with bounds
Another thing that we can do is to set some bounds into our solution, so as to contain our candidate solutions within a specific range. We can do this simply by passing a `bounds` parameter, of type `tuple`, when creating an instance of our swarm. Let's try this using the global-best PSO with the Rastrigin function (`rastrigin` in `pyswarms.utils.functions.single_obj`).

Recall that the Rastrigin function is bounded within `[-5.12, 5.12]`. If we pass an unbounded swarm into this function, then a `ValueError` might be raised. So what we'll do is to create a bound within the specified range. There are some things to remember when specifying a bound:

* A bound should be of type tuple with length 2. 
* It should contain two `numpy.ndarrays` so that we have a `(min_bound, max_bound)`
* Obviously, all values in the `max_bound` should always be greater than the `min_bound`. Their shapes should match the dimensions of the swarm.

What we'll do now is to create a 10-particle, 2-dimensional swarm. This means that we have to set our maximum and minimum boundaries with the shape of 2. In case we want to initialize an n-dimensional swarm, we then have to set our bounds with the same shape n. A fast workaround for this would be to use the `numpy.ones` function multiplied by a constant.

In [5]:
# Create bounds
max_bound = 5.12 * np.ones(2)
min_bound = - max_bound
bounds = (min_bound, max_bound)

In [6]:
%%time
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO with bounds argument
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=2, options=options, bounds=bounds)

# Perform optimization
cost, pos = optimizer.optimize(fx.rastrigin, iters=1000)

2021-11-13 11:37:55,145 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0      
2021-11-13 11:37:57,922 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0, best pos: [6.21204369e-10 5.74077510e-12]


Wall time: 2.79 s


## Basic Optimization with Arguments

Here, we will run a basic optimization using an objective function that needs parameterization.  We will use the ``single.GBestPSO`` and a version of the rosenbrock function to demonstrate

In [7]:
# import modules
import numpy as np

# create a parameterized version of the classic Rosenbrock unconstrained optimzation function
def rosenbrock_with_args(x, a, b, c=0):
    f = (a - x[:, 0]) ** 2 + b * (x[:, 1] - x[:, 0] ** 2) ** 2 + c
    return f

### Using Arguments

Arguments can either be passed in using a tuple or a dictionary, using the ``kwargs={}`` paradigm. First lets optimize the Rosenbrock function using keyword arguments.  Note in the definition of the Rosenbrock function above, there were two arguments that need to be passed other than the design variables, and one optional keyword argument, ``a``, ``b``, and ``c``, respectively

In [8]:
from pyswarms.single.global_best import GlobalBestPSO

# instatiate the optimizer
x_max = 10 * np.ones(2)
x_min = -1 * x_max
bounds = (x_min, x_max)
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
optimizer = GlobalBestPSO(n_particles=10, dimensions=2, options=options, bounds=bounds)

# now run the optimization, pass a=1 and b=100 as a tuple assigned to args

cost, pos = optimizer.optimize(rosenbrock_with_args, 1000, a=1, b=100, c=0)

2021-11-13 11:37:58,365 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=1.06e-6
2021-11-13 11:38:00,983 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.060314311737833e-06, best pos: [1.0008797  1.00181369]


It is also possible to pass a dictionary of key word arguments by using ``**`` decorator when passing the dict

In [9]:
kwargs={"a": 1.0, "b": 100.0, 'c':0}
cost, pos = optimizer.optimize(rosenbrock_with_args, 1000, **kwargs)

2021-11-13 11:38:01,177 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=7.1e-18
2021-11-13 11:38:03,804 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 7.101507021147657e-18, best pos: [1.         1.00000001]


Any key word arguments in the objective function can be left out as they will be passed the default as defined in the prototype. Note here, ``c`` is not passed into the function.

In [10]:
cost, pos = optimizer.optimize(rosenbrock_with_args, 1000, a=1, b=100)

2021-11-13 11:38:03,963 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=7.07e-18
2021-11-13 11:38:06,461 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 7.06939380187396e-18, best pos: [1.         1.00000001]
